# wr.data_api.redshift.connect_temp

This notebook attempts to make a connection to redshift with the `wr.data_api.redshift.connect_temp` method.

https://aws-data-wrangler.readthedocs.io/en/2.15.1/stubs/awswrangler.data_api.redshift.connect.html#awswrangler.data_api.redshift.connect

Doesn't have an example, I'm copying from my Redshift CHallenge notebook

In [1]:
import awswrangler as wr
import boto3
import psycopg2

In [2]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

QWWM-QLGL
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [3]:
session = boto3.session.Session(profile_name='Stellaralgo-DataScienceAdmin')
client = session.client('redshift')

CLUSTER = 'qa-app'
DBNAME = 'stlrcfl'

creds = client.get_cluster_credentials(                
    ClusterIdentifier = CLUSTER,
    DbUser = 'admin',
    DbName = DBNAME,
    DbGroups = ['admin_group'],
    AutoCreate=True
)

# create a connection & cursor and call the stored procedure
cnxn = psycopg2.connect(
    host = 'qa-app.ctjussvyafp4.us-east-1.redshift.amazonaws.com',
    port = 5439,
    user = creds['DbUser'],
    password = creds['DbPassword'],
    database = DBNAME
)

print(f"GOT CONNECTION TO DATABASE: {CLUSTER} {DBNAME}")

OperationalError: connection to server at "qa-app.ctjussvyafp4.us-east-1.redshift.amazonaws.com" (54.162.199.152), port 5439 failed: Operation timed out
	Is the server running on that host and accepting TCP/IP connections?


In [6]:
with cnxn.cursor() as cursor:
    cursor.execute("SELECT 1")
    print(cursor.fetchall())
cnxn.close()

AttributeError: 'RedshiftDataApi' object has no attribute 'cursor'

In [11]:
read_all_statement = f'SELECT * FROM datascience.ds.dummytable'

# We call the execute_sql func to reduce code duplication when reading/writing from/to Redshift
df = wr.data_api.redshift.read_sql_query(
                sql = read_all_statement, 
                con = cnxn
        )
df.head()

,playerid,dob,gamesplayed,injured,position,name,numassists,numgoals,pointpercentage
0,11,1988-08-04,20,false,RW,Dale,24,21,2.1
1,12,1985-06-05,20,false,C,Skip,15,36,2.5
2,13,1985-03-15,15,true,LW,Sanders,20,30,1.9
3,14,1983-02-20,20,false,LD,Patty,38,12,1.5
4,15,1987-08-04,18,false,RD,Reynolds,16,6,0.8
